# DREAM instrument view

This notebook is a simple example of how to use the instrument view for the DREAM instrument.

- The DREAM-specific instrument view is capable of slicing the data with a slider widget along a dimension (e.g. `tof`) by using the `dim` argument.
- There are also checkboxes to hide/show the different elements that make up the DREAM detectors.

In [ ]:
import scipp as sc
from ess import dream

## Load the data

We load a dataset from a Geant4 simulation (stored as a `.csv` file).
In each detector bank, the data is organised by `wire`, `strip`, `module`, `segment`, and `counter`.
The high resolution detector also has an additional `sector` dimension.

In [ ]:
dg = dream.io.load_geant4_csv(dream.data.get_path("data_dream0_new_hkl_Si_pwd.csv.zip"))
dg = dg["instrument"]  # Extract the instrument data

# Extract the events from nested data groups
dg = sc.DataGroup({key: detector["events"] for key, detector in dg.items()})

# Construct the pixel positions from event positions
for da in dg.values():
    da.coords["position"] = da.bins.coords["position"].bins.mean()

dg

In [ ]:
# Only plot half of the pixels to reduce html docs size
dg = dg["counter", 0]

## Full instrument view

We first histogram the data along the time-of-flight (`tof`) dimension,
making sure the same bins are used for all elements:

In [ ]:
tof_edges = sc.linspace("tof", 1.0e7, 1.0e8, 51, unit="ns", dtype=int)
data = dg.hist(tof=tof_edges)

We now use the `instrument_view` function to show the 3D view of the instrument pixels,
specifying that we wish to have a slider along the `tof` dimension:

In [ ]:
full_view = dream.instrument_view(data, dim="tof")
full_view

In [ ]:
full_view[2].controls["tof"]["slider"].value = 35

Note that it is possible to use any dimension for the slider instead of `tof`, such as `wavelength` (if present in the data).

## Displaying individual detector elements

It is also possible to view a single detector element, selecting e.g. `mantle` from the original data:

In [ ]:
mantle_view = dream.instrument_view(dg["mantle"].hist(wavelength=50), dim="wavelength")
mantle_view

In [ ]:
mantle_view[1].controls["wavelength"]["slider"].value = 43

The instrument view is designed to be flexible in terms of what it accepts as input.
This means that you can easily inspect, for example, a single module by using the usual slicing notation for data arrays.

Below, we display the first module in the backward end-cap detector:

In [ ]:
dream.instrument_view(dg["endcap_backward"]["module", 0].hist(tof=1))